In [0]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [0]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [0]:
openai = OpenAI()

In [0]:
# Let's just make sure the model is loaded
!ollama pull llama3.2
import ollama


In [0]:
# System prompt - defines the AI's behavior
SYSTEM_PROMPT = """You are a helpful cooking assistant that provides ingredient lists for recipes.
Format your response as clean markdown with this structure:

# [Dish Name]
**Serves:** [number] people  
**Cook Time:** [estimated time]

## Shopping List
- [ ] [amount] [unit] [ingredient]
- [ ] [amount] [unit] [ingredient]

Guidelines:
- Use common grocery store measurements (cups, lbs, oz, pieces, cans, etc.)
- Round to practical shopping amounts (1.5 lbs instead of 1.47 lbs)
- Group similar items when logical (all spices together)
- Include pantry staples only if they're essential (salt, oil, etc.)
- Assume basic seasonings are available unless recipe-specific
- For produce, specify size when important (large onion, medium tomatoes)
- Keep optional items at the end of similar item groups or end of the list"""

In [0]:
def get_recipe_openai(dish_name: str, num_people: int):
    """Get scaled recipe ingredients using system and user prompts"""

    user_prompt = f"Give me the ingredients needed to make {dish_name} for {num_people} people."
    
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=400
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"❌ Error: Failed to get recipe - {str(e)}"

In [0]:
OLLAMA_MODEL = "llama3.2"

In [0]:
def get_recipe_ollama(dish_name: str, num_people: int):
    """Get recipe using Ollama API"""
    user_prompt = f"Give me the ingredients needed to make {dish_name} for {num_people} people."
    
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]
    
    try:
        response = ollama.chat(model=OLLAMA_MODEL, messages=messages)
        return response['message']['content']
    except Exception as e:
        return f"❌ Ollama Error: {str(e)}"

In [0]:
def print_shopping_list(recipe_markdown):
    """Print the markdown response"""
    display(Markdown(recipe_markdown))

In [0]:
print("🍳 Recipe Scaler & Grocery List Maker")
print("=" * 40)
    
ai_service_choice = input("\nChoose AI service (1 for OpenAI, 2 for Ollama): ").strip()

dish = input("What dish do you want to make? ")
num_people = int(input("How many people? "))
    
print(f"\n🔍 Getting recipe for {dish}...")
    
# Get and display recipe
if ai_service_choice == '1':
    print("Using OpenAI API...")
    recipe_markdown = get_recipe_openai(dish, num_people)
else:
    print("Using Ollama (local)...")
    recipe_markdown = get_recipe_ollama(dish, num_people)

print_shopping_list(recipe_markdown)